# Statistiques descriptives du jeu de données

### Initialisation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(r'C:\Users\tlp\Downloads\fullcorpus20210725_DATE.csv', encoding = 'ISO-8859-1').drop('Unnamed: 0',axis=1)
df.replace({'Ã©': 'é', 'Ã¨': 'è', 'Ãª': 'ê', 'Ã´': 'ô','Ã': 'à','Â': '', 'â': "'", 'à ': 'à'}, regex=True, inplace=True)
df["CJournal"].replace("Autre", "Echos", inplace=True)
print(df.shape)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\tlp\\Downloads\\fullcorpus20210725_DATE.csv'

### Proportion d'articles par journaux

In [ ]:
# Supposons que df["CJournal"].value_counts() retourne un objet Series avec les comptages
comptages = df["CJournal"].value_counts()

# Créer le graphique pie avec Seaborn
plt.figure(figsize=(8, 8))
sns.set(style="whitegrid")
plt.pie(comptages, labels=comptages.index, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Distribution des journaux', fontsize=18, loc='center', pad=20, weight='bold')
plt.show()

In [ ]:
pd.DataFrame(comptages).reset_index(names="Journal").rename(columns={"CJournal":"Articles"})

### Distribution temporelle par journaux

In [ ]:
# Convertir la colonne "Date" en type datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filtrer les données pour les années avant et après 2015
df_before_2015 = df[df['Date'].dt.year < 2015]
df_after_2015 = df[df['Date'].dt.year >= 2015]

# Créer des DataFrames pour les graphiques par mois et par année
dfs = [(df, ''), (df_after_2015, 'Après 2015')]

# Tracer les graphiques pour chaque période
for df_period, title in dfs:
    # Graphique par mois
    df_monthly = df_period["Date"].groupby([df_period['Date'].dt.year, df_period['Date'].dt.month]).count()

    plt.figure(figsize=(12, 6))
    df_monthly.plot(kind='line', marker='o', linewidth=2)
    plt.title(f'Évolution temporelle du nombre de journaux par mois {title}')
    plt.xlabel('Mois')
    plt.ylabel('Nombre de journaux')
#     plt.legend(title='Journal')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Graphique par année
    df_yearly = df_period.groupby([df_period['Date'].dt.year, df_period['CJournal']]).size().unstack(fill_value=0)
    plt.figure(figsize=(12, 6))
    df_yearly.plot(kind='line', marker='o', linewidth=2)
    plt.title(f'Évolution temporelle du nombre de journaux par année {title}')
    plt.xlabel('Année')
    plt.ylabel('Nombre de journaux')
    plt.legend(title='Journal')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

Il n'y a qu'après 2015 qu'on peux en tirer quelque chose

In [ ]:
# Convertir la colonne "Date" en type datetime
df['Date'] = pd.to_datetime(df['Date'])
df["Annee"] = df['Date'].dt.year
df["Mois"] = df['Date'].dt.month


# Graphique par mois
df_monthly = df[["Annee","Mois","CJournal"]].groupby(['Annee','Mois']).count().rename(columns={"CJournal":"Total"})

plt.figure(figsize=(20, 15))
df_monthly.plot(kind='line', marker='o', linewidth=2)
plt.title(f"Évolution du nombre d'articles par mois")
plt.xlabel('Mois')
plt.ylabel('Nombre de journaux')


# # Obtenir les étiquettes des mois au format 'mm-yyyy' en utilisant une double boucle for
# labels = []
# for year, month in np.array(df_monthly.reset_index()[["Annee","Mois"]]):
#     labels.append(f"{month:02d}-{year}")
# #         if year == 2015 and month < 2:
# #             continue  # Ignorer janvier 2015
# #         if year == 2020 and month > 12:
# #             break  # Sortir de la boucle après décembre 2020
# #         labels.append(f"{month:02d}-{year}")

# # Définir les étiquettes des mois sur l'axe des abscisses
# plt.xticks(ticks=range(len(labels)), labels=labels, rotation=45)

plt.grid(True)
# plt.tight_layout()
plt.show()

# Graphique par année
df_yearly = df.groupby([df['Date'].dt.year, df['CJournal']]).size().unstack(fill_value=0)
plt.figure(figsize=(20, 15))
df_yearly.plot(kind='line', marker='o', linewidth=2)
plt.title(f"Évolution du nombre d'articles par journaux par an")
plt.xlabel('Année')
plt.ylabel('Nombre de journaux')
plt.legend(title='Journal')
plt.grid(True)
# plt.tight_layout()
plt.show()

### Longueur moyenne des articles

In [ ]:
# Calculer la longueur de chaque article (nombre de mots)
df['Longueur_article'] = df['Texte'].apply(lambda x: len(x.split()))

# Extraire l'année à partir de la colonne "Date"
df['Année'] = df['Date'].dt.year

# Agréger les données par journal et par année
df_aggregated = df.groupby(['CJournal', 'Année']).agg({'Longueur_article': 'mean'}).reset_index()

# Tracer l'évolution de la longueur moyenne des articles par journal et par année
journals = df_aggregated['CJournal'].unique()
plt.figure(figsize=(12, 6))

for journal in journals:
    df_journal = df_aggregated[df_aggregated['CJournal'] == journal]
    plt.plot(df_journal['Année'], df_journal['Longueur_article'], marker='o', label=journal)

plt.title('Évolution de la longueur moyenne des articles en nombre de mot par journal et par année')
plt.xlabel('Année')
plt.ylabel('Longueur moyenne des articles')
plt.legend(title='Journal')
plt.grid(True)
plt.tight_layout()
plt.show()